# Analisis de variables categoricas de cada cluster

Ya contamos con los 12 clusters de nuestro modelo de K-means sin embargo cluster 1,2,3 ... 12 no aporta nada al negocio por ende toca caracterizar cada uno de ellos para posteriormente etiquetar los mismos

En este jupiter nos centraremos en analizar las variables categoricas de los 12 clusters que tenemos

In [13]:
# Importamos las librerías necesarias
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

df_customer = pd.read_csv("dataset/segmented_customers.csv")
df_customer['Cluster'] = df_customer['Cluster'].astype('category')
df_customer.head()

,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Payment Method,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,Cluster
0,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Credit Card,Express,Yes,Yes,14,Venmo,Fortnightly,2
1,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Bank Transfer,Express,Yes,Yes,2,Cash,Fortnightly,1
2,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Cash,Free Shipping,Yes,Yes,23,Credit Card,Weekly,2
3,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,PayPal,Next Day Air,Yes,Yes,49,PayPal,Weekly,2
4,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Cash,Free Shipping,Yes,Yes,31,PayPal,Annually,2


In [14]:
# Verificamos el tipo de dato de cada columna
# df_customer.dtypes

Vamos a crear una función para realizar la prueba de Chi-cuadrado y determinar si una variable es distintiva ; la prueba compara la distribución de una variable categórica dentro de un clúster  vs. la distribución de la misma variable en el resto del dataset.

In [ ]:
def is_distinctive_feature(dataframe, cluster_num, column_name, alpha=0.05):
    """
    Realiza la prueba de Chi-cuadrado para determinar si una variable categórica
    es una característica distintiva para un clúster específico.
    """
    # Creamos un DataFrame para el clúster actual
    cluster_df = dataframe[dataframe['Cluster'] == cluster_num]
    # Creamos un DataFrame para todos los demás clústeres
    rest_df = dataframe[dataframe['Cluster'] != cluster_num]

    # Contamos las frecuencias para el clúster actual
    cluster_counts = cluster_df[column_name].value_counts(normalize=True).sort_index()
    # Contamos las frecuencias para el resto de los clústeres
    rest_counts = rest_df[column_name].value_counts(normalize=True).sort_index()

    # Combinamos las etiquetas de ambas series para asegurar que la tabla de contingencia
    # tenga las mismas columnas (categorías de la variable)
    all_categories = pd.concat([cluster_counts, rest_counts]).index.unique()

    # Creamos la tabla de contingencia
    # Si una categoría no existe en un grupo, su conteo es 0.
    contingency_table = pd.DataFrame({
        'Cluster': [cluster_counts.get(cat, 0) for cat in all_categories],
        'Resto': [rest_counts.get(cat, 0) for cat in all_categories]
    }, index=all_categories)
    
    # Realizamos la prueba de Chi-cuadrado
    chi2, p_value, _, _ = chi2_contingency(contingency_table)

    # Retornamos si el resultado es significativo (p_value < alpha)
    return p_value < alpha

Para automatizar todo el proceso y quedarnos solo con las variables que diferencian a un cluster del resto hemos creado la siguiente funcion para solo obtener esos resultados y mejorar la legibilidad de nuestros resultados

In [18]:
def find_distinctive_categorical_features(df, cluster_id):
    """
    Busca y presenta solo las variables categóricas que son
    estadísticamente distintivas para un clúster dado,
    usando la prueba de Chi-cuadrado.

    Args:
        df (pd.DataFrame): El DataFrame que contiene la columna 'Cluster'.
        cluster_id (int): El ID del clúster a analizar.
    """
    # Filtramos el DataFrame para el clúster actual
    current_cluster_df = df[df['Cluster'] == cluster_id]
    cluster_count = len(current_cluster_df)

    print("\n" + "="*80)
    print(f"ANÁLISIS DE VARIABLES CATEGÓRICAS DISTINTIVAS DEL CLÚSTER {cluster_id}")
    print(f"Número de elementos en este clúster: {cluster_count}")
    print("="*80)

    # Obtenemos la lista de variables categóricas
    categorical_cols = df.select_dtypes(include=['object']).columns
    
    distinctive_features_found = False

    # Iteramos sobre cada variable categórica
    for col in categorical_cols:
        if not current_cluster_df.empty:
            is_distinctive = is_distinctive_feature(df, cluster_id, col)
            
            if is_distinctive:
                distinctive_features_found = True
                
                # Obtenemos el valor más frecuente (moda)
                most_frequent_value = current_cluster_df[col].mode()[0]
                
                # Obtenemos la frecuencia de la moda
                frequency = len(current_cluster_df[current_cluster_df[col] == most_frequent_value])
                
                # Imprimimos los resultados de la variable distintiva
                print(f"   Variable: {col}")
                print(f"   Moda (valor más frecuente): '{most_frequent_value}'")
                print(f"   Frecuencia de la Moda: {frequency}")
                print("-" * 40)
    
    if not distinctive_features_found:
        print("   Este clúster no tiene características categóricas distintivas.")
        print("   Considera analizar las variables numéricas para caracterizarlo.")
        print("-" * 40)

In [20]:
# Llama a la función para cada clúster, desde 0 hasta 11
for i in range(12):
    find_distinctive_categorical_features(df_customer, i)


ANÁLISIS DE VARIABLES CATEGÓRICAS DISTINTIVAS DEL CLÚSTER 0
Número de elementos en este clúster: 140
   Este clúster no tiene características categóricas distintivas.
   Considera analizar las variables numéricas para caracterizarlo.
----------------------------------------

ANÁLISIS DE VARIABLES CATEGÓRICAS DISTINTIVAS DEL CLÚSTER 1
Número de elementos en este clúster: 159
   Este clúster no tiene características categóricas distintivas.
   Considera analizar las variables numéricas para caracterizarlo.
----------------------------------------

ANÁLISIS DE VARIABLES CATEGÓRICAS DISTINTIVAS DEL CLÚSTER 2
Número de elementos en este clúster: 953
   Este clúster no tiene características categóricas distintivas.
   Considera analizar las variables numéricas para caracterizarlo.
----------------------------------------

ANÁLISIS DE VARIABLES CATEGÓRICAS DISTINTIVAS DEL CLÚSTER 3
Número de elementos en este clúster: 310
   Este clúster no tiene características categóricas distintivas.
   C

<u>Conclusiones</u>

Que ninguna de las variables categoricas sea una caracteristica distintiva para ninguno de los 12 clusteres nos dice algo muy importante: la segmentacion no se basa en estas variables. La forma en que los clientes se agruparon no tiene que ver con su genero, su ubicacion, o el tipo de producto que compran

Esto no es un fracaso del modelo. Es un hallazgo. Simplemente significa que la "historia" de los clientes no esta en los datos categoricos, sino que está oculta en las variables numericas pero eso es algo que veremos en el siguiente notebook